# Asymmetric Encryption

Reminder: In the asymmetric encryption schemes the parties use ***different*** keys, that are mathematically ***related*** to each other.

## *<font color=" #6495ED">Exercise</font>*

 - Why asymmetric encryption is useful?
 - Give a few examples where it can be used?

## RSA

RSA, is an asymmetric encryption algorithm by Ron Rivest, Adi Shamir, and Leonard Adleman. It was published in 1977. It's security is based on the hardness of factorization problem. However, now it has its own problem, called the RSA problem. RSA is slow, and is not used for encrypting large data, but it's mostly used to encrypt the symmetric key that is used for encryption.


 * $p, q$, two big prime numbers (private, chosen)
 * $n = pq$, $φ(n) = (p-1)(q-1)$   (public, calculated)
 * $e$, with $gcd(φ(n), e) = 1,  1 < e < φ(n)$	(public, chosen)
 * $d = e - 1$ mod $φ(n)$	(private, calculated)
 * $E(M) = M^e \mod n$
 * $D(M) = M^d \mod n$
 * $D(E(M)) = M^{ed} \mod n = M$

## *<font color=" #6495ED">Exercise</font>*

 - How to test if a number is prime?

## RSA EXAMPLE

 - p = 5; q = 11 => n = 55
 - φ(n) = 40
 - e = 3 => d = 27
  - Because ed = 1 mod φ(n)
 - Public key: (e, n)
 - Private key: (d, n)
 - Encryption
  - M = 2
 - Encryption(M) = $ M^e\mod n$  = $2^3\mod n$ = 8
 - Decryption(8) = $ M^d\mod n$  = $8^{27} \mod n$ = 2

## *<font color=" #6495ED">Exercise</font>*

 - Perform textbook RSA encryption and decryption using the values from the example

In [ ]:
p = 5
q = 11
n = p * q
e = 3
d = 27
m = 2

## *<font color=" #b74138">Solution</font>*

In [ ]:
e * d

In [ ]:
e * d % ((p-1)*(q-1))

In [ ]:
c = m ** e
print(c)

In [ ]:
c = m ** e % n
print(c)

In [ ]:
c = pow(m, e, n)
print(c)

In [ ]:
c ** d

In [ ]:
m_dec = c ** d % n
print(m_dec)

In [ ]:
m_dec = pow(c, d, n)
print(m_dec)

### Signing

The purpose of signatures is to prove the "creator" of a message. To perform this task we use the private key, since only the owner of the keypair is aware of the private key. Everyone with the knowledge of the public key can verify the signature.

## *<font color=" #6495ED">Exercise</font>*

 - Create textbook RSA signature on message (m=2). Use the private key (d), instead of the public key (e) to generate the signature. Use the public key (e) to verify the signature.

## *<font color=" #b74138">Solution</font>*

In [ ]:
p =5
q = 11
n = p * q
e = 3
d = 27
m = 2

In [ ]:
sig = pow(m, d, n)
print(sig)

In [ ]:
m = pow(sig, e, n)
print(m)

### Generating RSA keypairs using OpenSSL

To generate keys, use the following instructions:

```bash
 openssl genrsa -out private_key.pem 2048
 openssl pkcs8 -topk8 -inform PEM -outform DER -in private_key.pem -out private_key.der -nocrypt
 openssl rsa -in private_key.pem -pubout -outform DER -out public_key.der
 ```

In [ ]:
# import key from a file. E.g., previously generated by OpenSSL
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

with open("private_key.pem", "rb") as key_file:
     private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend())
public_key = private_key.public_key()

In [ ]:
# Generate a 2048 bit private key
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend())
# to get the public key
public_key = private_key.public_key()

In [ ]:
2 ** 16 +1 

In [ ]:
print(bin(2**16 + 1))
print(bin(2**1 + 1))

## *<font color=" #6495ED">Exercise</font>*

 - What's wrong with textbook RSA?

### It's all about padding and randomness

### Optimal Asymmetric Encryption Padding (OAEP)

Textbook RSA is not IND-CPA secure, therefore we use Optimal Asymmetric Encryption Padding (OAEP).

<img src="include/RSA_OAEP.png">

image souce: wikipedia

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

message = b"Y"*127
ciphertext = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA1()),
        algorithm=hashes.SHA1(),
        label=None))

In [ ]:
print(ciphertext)

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA1()),
        algorithm=hashes.SHA1(),
        label=None))

In [ ]:
print(plaintext)

### Probabilistic Signature Scheme (PSS)

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

message = b"Message to be signed"

In [ ]:
# signature creation
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256())

In [ ]:
# signature verification
public_key.verify(
    signature,
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256())

## *<font color=" #6495ED">Exercise</font>*

 - Compare the speed of encryption using RSA-2048 and AES-128.

## *<font color=" #b74138">Solution</font>*

In [ ]:
import os
import time
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

message = b"Y"*200
aes_key = os.urandom(16)
ctr = os.urandom(16)

def rsa_2048_enc(message):
    public_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA1()),
            algorithm=hashes.SHA1(),
            label=None))

    
def aes_128_enc(message):
    cipher = Cipher(algorithms.AES(aes_key), modes.CTR(ctr), backend=default_backend())
    encryptor = cipher.encryptor()
    ctr_ct = encryptor.update(message) + encryptor.finalize()

In [ ]:
loops = 50000

s = time.time()

for i in range(loops):
    rsa_2048_enc(message)

e = time.time()
print("RSA: Encrypted {} messages in {} seconds".format(loops, e-s))


s = time.time()

for i in range(loops):
    aes_128_enc(message)

e = time.time()
print("AES: Encrypted {} messages in {} seconds".format(loops, e-s))

These tests give an idea of the speed difference between RSA and AES. However, our benchmark does not reflect the whole picture. To get a better view run:

``` bash
 $ openssl speed rsa aes
```

## Diffie–Hellman key exchange

Diffie-Hellman is a key exchange protocol and an example of an asymmetric cryptography. Named after it's inventors  Whitfield Diffie and Martin Hellman in 1976. The security of DH key exchange relies on the hardness of discrete logarithm.

 - Alice: generate private key ($a$). Calculate public key $A = (g ^ a)$ mod $p$
 - Bob: generate private key ($b$). Calculate public key $B = (g ^ b)$ mod $p$
 
 
 - Alice -> Bob: Alice sends her public key ($A$) to Bob
 - Bob -> Alice: Bob send his public key ($B$) to Alice


 - Alice and Bob: Calculate shared key K = $g ^ {ab}$ mod $p$
 - $(g ^ a) ^ b = (g ^ b) ^ a = g ^ {ab}$

## *<font color=" #6495ED">Exercise</font>*

 - Perform basic DH key exchange using the values below to create a shared key ($g^{ab}$)

In [ ]:
g = 2
p = 19
a = 7
b = 8

## *<font color=" #b74138">Solution</font>*

In [ ]:
gab = pow(pow(g, a, p), b, p) # (g^a) ^ b
print(gab)

In [ ]:
gba = pow(pow(g, b, p), a, p) # (g^b) ^ a
print(gba)

In [ ]:
assert gab == gba

### Performing DH key exchange with key derivation

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

parameters = dh.generate_parameters(generator=2, key_size=2048,
                                    backend=default_backend())

private_key_a = parameters.generate_private_key()  # a
private_key_b = parameters.generate_private_key()  # b

In [ ]:
public_key_a = private_key_a.public_key() # g^a
public_key_b = private_key_b.public_key() # g^b

In [ ]:
shared_key_b = private_key_b.exchange(public_key_a) # (g^a) ^ b = g ^ ab

In [ ]:
shared_key_a = private_key_a.exchange(public_key_b) # (g^a) ^ b = g ^ ab

In [ ]:
assert shared_key_a == shared_key_b # (g^a)^b == (g^b)^a == g^ab == g^ba

In [ ]:
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'shared DH key',
    backend=default_backend()).derive(shared_key_a)

In [ ]:
len(derived_key)

## Elliptic-curve cryptography (ECC)

ECC is another asymmetric approach that is based on elliptic curves over finite fields. ECC is a more recent approach compared to RSA. They key sizes in ECC are smaller than RSA. As a result it can consume less resources and is popular in environments with constrained resources such as IoT devices and sensors.

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec

private_key = ec.generate_private_key(ec.SECP256K1(), default_backend())

data = b"Some data to be signed"
signature = private_key.sign(data, ec.ECDSA(hashes.SHA256()))

In [ ]:
public_key = private_key.public_key()

In [ ]:
public_key.verify(signature, data, ec.ECDSA(hashes.SHA256()))

In [ ]:
public_key.verify(signature+b"1", data, ec.ECDSA(hashes.SHA256()))